In [ ]:
import json
import math
import os
import time
from functools import partial
from urllib.parse import urlencode
from fastparquet import write
import faiss
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

import requests
from catboost import CatBoostRanker, Pool
from faiss import read_index, write_index
from sentence_transformers import SentenceTransformer

In [ ]:
# %pip install -r ./requirements.txt.

In [ ]:
def download_file(pk, local_name):
    base_url = "https://cloud-api.yandex.net/v1/disk/public/resources/download?"

    final_url = base_url + urlencode(dict(public_key=pk))
    response = requests.get(final_url)
    download_url = response.json()["href"]

    download_response = requests.get(download_url)
    with open(local_name, "wb") as f:
        f.write(download_response.content)
        print(f"File {local_name} downloaded")

def download_from_yandex_disk():
    files = {
        "features.parquet": "https://disk.yandex.ru/d/W_qJitz4dZGzAg",
        "videos.parquet": "https://disk.yandex.ru/d/JXz-oDfKFgm2Dw",
        "automarkup.parquet": "https://disk.yandex.ru/d/vP0FzQHdtxsz4Q",
        "manualmarkup.csv": "https://disk.yandex.ru/d/hDztN1rgW0JNjw",
    }

    filenames, filenames_to_delete = files.keys(), []
    for filename in filenames:
        if os.path.exists(filename):
            filenames_to_delete += [filename]
    for filename in filenames_to_delete:
        del files[filename]

    print(f"Will be download {len(files)} files")
    for filename, link in files.items():
        print(f"{filename} downloading in progress")
        download_file(link, filename)
        print(f"{filename} downloaded")

In [ ]:
download_from_yandex_disk()

In [ ]:
seed = 42
np.random.seed(seed)

In [ ]:
candidates = pd.read_parquet(
    "videos.parquet", engine="fastparquet", columns=["video_id", "video_title"]
)

In [ ]:
features_parquet1 = pd.read_parquet("features.parquet")
features_parquet1.drop_duplicates(subset=['video_id'], inplace=True)
features_parquet1['asd'] = features_parquet1['v_year_views']+\
        features_parquet1['v_month_views']+\
        features_parquet1['v_week_views']+\
        features_parquet1['v_likes']+\
        features_parquet1['v_dislikes']+\
        features_parquet1['total_comments']

In [ ]:
new_cd = candidates[(candidates['video_title'] != '') & (candidates['video_title'] != 'Без названия') & (candidates['video_title'].str.len() > 5)]
new_cd.drop_duplicates(inplace=True)
new_cd

In [ ]:
full_df = new_cd.merge(
    features_parquet1[features_parquet1['asd'] > 5], how="right", left_on="video_id", right_on="video_id"
)
full_df

In [ ]:
corpus = full_df["video_title"].apply(lambda x: str(x).lower().strip()).values
video_ids = full_df["video_id"].values

In [ ]:
st_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cpu")

In [ ]:
d = 312
cpu_index = faiss.IndexFlatL2(d)
cpu_index.is_trained, cpu_index.ntotal

In [ ]:
ind2videoid = {ind: video_id for ind, video_id in enumerate(video_ids)}
with open("ind2videoid-train.json", "w+") as f:
    json.dump(ind2videoid, f, indent=4)

In [ ]:
batch_size = 1000000
num_batches = math.ceil(len(corpus) / batch_size)

In [ ]:
try:
    for i in range(num_batches):
        # формируем батч
        start, end = i * batch_size, (i + 1) * batch_size
        corpus_batch = corpus[start:end]

        # считаем вектора для всех предложений в батче
        embeddings = st_model.encode(
            corpus_batch, show_progress_bar=True
        )#batch_size=num_batches,

        # добавляем новые батч векторов в индекс и сохраняем его
        cpu_index.add(embeddings)
        write_index(cpu_index, "candidates.index")
        print(f"batch: {i + 1} / {num_batches}, vectors: {cpu_index.ntotal}")
        # чистим ОЗУ
        del embeddings
    # write_index(cpu_index, "candidates.index")
except KeyboardInterrupt:
    print("Остановлено пользователем")
    try:
        del embeddings
    except:
        pass

In [ ]:
automarkup = pd.read_parquet("automarkup.parquet", engine="fastparquet")

In [ ]:
automarkup = automarkup[~automarkup["query"].isna()]
automarkup["query"] = automarkup["query"].apply(lambda x: x.lower().strip())

In [ ]:
n = 1000
top_n = automarkup["query"].value_counts()[: int(2 * n)].index.to_list()
other = np.array(automarkup["query"].value_counts()[int(2 * n) :].index.to_list())
random_n = np.random.choice(other, size=n, replace=False).tolist()
queries = top_n + random_n
query2ind = {q: i for i, q in enumerate(queries)}

In [ ]:
use_formed_candidates = False

In [ ]:
qembeddings = st_model.encode(queries, show_progress_bar=True)#batch_size=1000, 

In [ ]:
search_cpu_index = read_index("candidates.index")
search_cpu_index.is_trained, search_cpu_index.ntotal

In [ ]:
topk = 300
distance, faiss_ind = search_cpu_index.search(qembeddings, topk)

In [ ]:
from tqdm import tqdm
dtype = [('distance', float), ('index', int)]
generated_cand_name = "generated_candidates.parquet"
# distance, faiss_ind = search_cpu_index.search(qembeddings, topk)
generated_cand = {"query": [], "video_id": [], "top":[]}
pbar = tqdm(total=len(queries))
for i, q in enumerate(queries):
    vids = faiss_ind[i]
    dist = distance[i]
    percentel = np.percentile(dist,2.5)
    sorted_output = np.sort(np.array([(i,j) for i,j in zip(dist, faiss_ind[i])], dtype=dtype), order='distance')
    generated_cand["video_id"] += [ind2videoid[str(v)] for v in vids]
    generated_cand["query"] += [q] * len(vids)
    if int(distance[i][0]) == 0:
        generated_cand["top"] += [1 if d<5 else 0 for d, i in sorted_output]
    else:
        mulp_dist =dist[0]*2
        generated_cand["top"] += [1 if d<percentel and d<mulp_dist else 0 for d, i in sorted_output]
    pbar.update(1)
pbar.close()

generated_cand = pd.DataFrame(generated_cand)

generated_cand.to_parquet(generated_cand_name, engine="fastparquet")
write(generated_cand_name, generated_cand)


# Формирование таргета по авторазметке

In [ ]:
automarkup["target"] = [1] * automarkup.shape[0]
candidates_with_target = generated_cand.merge(
    automarkup[["query", "video_id", "target"]],
    how="left",
    left_on=["query", "video_id"],
    right_on=["query", "video_id"],
)
candidates_with_target["target"] = candidates_with_target["target"].fillna(0)

In [ ]:
features_parquet = pq.ParquetFile("features.parquet")
featuresArr, filter_date = [], "2023-05-02"

for batch in features_parquet.iter_batches():
    tmp = batch.to_pandas()
    featuresArr.append( tmp[tmp["report_date"] == filter_date])

features = pd.concat(featuresArr, axis=0)
features = features.drop(
    ["v_channel_reg_datetime", "v_pub_datetime","v_channel_type", "v_category"], axis=1 # "v_channel_type", "v_category",
)

# Clickbait

In [ ]:
df = pd.read_csv('./manualmarkup.csv')
df_clickbait = df.loc[df['sentiment'] == "сlickbait"]
df_exact =  df.loc[df['sentiment'] == "could_be_better"]
clickbait_dataset = []
for batch in features_parquet.iter_batches():
    clickbait_dataset.append(df_clickbait.merge(batch.to_pandas(), on='video_id',how='inner'))
    clickbait_dataset.append(df_exact.merge(batch.to_pandas(), on='video_id',how='inner'))
clickbait_dataset = pd.concat(clickbait_dataset,ignore_index=True)
clickbait_dataset.drop_duplicates(subset='video_id', inplace=True)
clickbait_dataset = pd.get_dummies(clickbait_dataset,columns=['sentiment'],dtype=int,drop_first=True)
clickbait_dataset = clickbait_dataset.drop(columns=[
    'query','query_date','report_date','v_channel_reg_datetime','v_pub_datetime','video_id','v_category','v_channel_type'])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold

y = clickbait_dataset['sentiment_сlickbait']
x = clickbait_dataset.drop(columns=['sentiment_сlickbait'])
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)
clickbait_dataset_cols = np.array(clickbait_dataset.drop(columns=['sentiment_сlickbait']).columns)

In [ ]:
from catboost import CatBoostClassifier
params_clf = {
    'depth': 9, 
    'iterations': 100, 
    'learning_rate': 0.03
}
clickbait_clf = CatBoostClassifier(verbose=False, **params_clf)
clickbait_clf.fit(X_train, y_train, plot=True)

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = clickbait_clf.predict(X_val)
cm = confusion_matrix(y_val,ypred)
print(cm)
from sklearn.metrics import classification_report
report = classification_report(y_val, ypred)
print(report)
clickbait_clf.save_model("clickbait_clf.ckpt")

# Percent of watch

In [ ]:
automarkup_percent = automarkup.drop(columns=[
    'query','datetime','is_authorized','position','position','vtop','comment','channel','tv_show','season','emotion'])#, 'target'
percent_of_watch = automarkup_percent.groupby(['video_id']).median()
percent_of_watch['percent_of_watch'] = (percent_of_watch['watchtime'] * 1000) / percent_of_watch['duration']
percent_of_watch = percent_of_watch.drop(columns=['duration','watchtime'])

# Формирование датасета с признаками

In [ ]:
full_df = candidates_with_target.merge(
    features, how="inner", left_on="video_id", right_on="video_id"
)
# del features
full_df = full_df.drop("report_date", axis=1)
full_df = full_df.drop_duplicates()

In [ ]:
is_clickbait = clickbait_clf.predict(full_df[clickbait_dataset_cols])
full_df['clickbait'] = is_clickbait

In [ ]:
full_df = full_df.merge(
    percent_of_watch, how="left", left_on="video_id", right_on="video_id"
)

In [ ]:
groups_to_drop = []
full_df["group_id"] = full_df.groupby(["query"]).ngroup()
for group in full_df["group_id"].unique():
    part_df = full_df[full_df["group_id"] == group]
    target_sum = part_df["target"].values.sum()
    if target_sum <= 0:
        groups_to_drop += [group]
full_df = full_df[~full_df["group_id"].isin(groups_to_drop)]

In [ ]:
groups = pd.Series(full_df["group_id"].unique())
permutation = groups.sample(frac=1, random_state=seed)
train_groups, val_groups, test_groups = np.split(
    permutation, [int(0.75 * len(permutation)), int(0.90 * len(permutation))]
)
groups.shape, permutation.shape

In [ ]:
train_df = full_df[full_df["group_id"].isin(train_groups)]
val_df = full_df[full_df["group_id"].isin(val_groups)]
test_df = full_df[full_df["group_id"].isin(test_groups)]

In [ ]:
train_df = train_df.sort_values("group_id")
val_df = val_df.sort_values("group_id")
test_df = test_df.sort_values("group_id")

In [ ]:
metainfo_columns = ["query", "video_id", "target", "group_id"]

X_train = train_df.drop(metainfo_columns, axis=1)
y_train, g_train = train_df["target"], train_df["group_id"]

X_val = val_df.drop(metainfo_columns, axis=1)
y_val, g_val = val_df["target"], val_df["group_id"]

X_test = test_df.drop(metainfo_columns, axis=1)
y_test, g_test = test_df["target"], test_df["group_id"]

In [ ]:
train = Pool(
    data=X_train.values,
    label=y_train.values,
    group_id=g_train.values,
    feature_names=X_train.columns.to_list(),
)

val = Pool(
    data=X_val.values,
    label=y_val.values,
    group_id=g_val.values,
    feature_names=X_val.columns.to_list(),
)

test = Pool(
    data=X_test.values,
    label=y_test.values,
    group_id=g_test.values,
    feature_names=X_test.columns.to_list(),
)

# Обучение модели

In [ ]:
task_type = "CPU"#
metric_period = 250

parameters = {
    # "task_type": task_type,
    "verbose": False,
    "random_seed": seed,
    "loss_function": "QueryRMSE",
    "learning_rate": 0.001,
    "l2_leaf_reg": 30,
    "iterations": 2000,
    "max_depth": 10,
    # "task_type":"GPU",
    # "devices":'0'
}

In [ ]:
model = CatBoostRanker(**parameters)
model = model.fit(
    train, eval_set=val, plot=True, use_best_model=True, metric_period=metric_period
)
model.save_model("ranker.ckpt")

In [ ]:
import matplotlib.pyplot as plt
feature_importance = model.get_feature_importance(train)
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 10))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_train.columns)[sorted_idx])
plt.title('Feature Importance')


In [ ]:
def _metrics_at(at, model, pool, metric="NDCG"):
    metric = metric + f":top={at}"
    eval_metrics = model.eval_metrics(pool, metrics=[metric])
    best_metrics = {}
    for key in eval_metrics.keys():
        best_metrics[key] = eval_metrics[key][model.best_iteration_]
    return best_metrics

metrics_train_at = partial(_metrics_at, model=model, pool=train)

metrics_val_at = partial(_metrics_at, model=model, pool=val)

metrics_test_at = partial(_metrics_at, model=model, pool=test)